In [1]:
!pip install tokenizers
!pip install datasets



Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\bbdnet3197\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\bbdnet3197\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from datasets import load_dataset
ds_ibo= load_dataset("shmuhammad/AfriSenti-twitter-sentiment", "ibo")

C:\Users\bbdnet3197\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_texts = ds_ibo['train']['tweet']
train_labels = ds_ibo['train']['label']
test_texts = ds_ibo['test']['tweet']
test_labels = ds_ibo['test']['label']

In [4]:
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()
tokenizer.train_from_iterator(train_texts, vocab_size=5000, min_frequency=2)


In [5]:
def tokenize_to_subwords(text_list):
    return [" ".join(tokenizer.encode(text).tokens) for text in text_list]

train_subwords = tokenize_to_subwords(train_texts)
test_subwords = tokenize_to_subwords(test_texts)


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_stf = TfidfVectorizer(max_features=5000)
X_train = vectorizer_stf.fit_transform(train_subwords)
X_test = vectorizer_stf.transform(test_subwords)


In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

X_train = X_train.toarray()
X_test = X_test.toarray()
y_train = np.array(train_labels)
y_test = np.array(test_labels)

model = tf.keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(3, activation='softmax')  # 3 sentiment classes
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)


Epoch 1/10
287/287 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.5392 - loss: 0.9586 - val_accuracy: 0.7010 - val_loss: 0.7183
Epoch 2/10
287/287 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.8106 - loss: 0.4773 - val_accuracy: 0.7000 - val_loss: 0.8093
Epoch 3/10
287/287 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.8815 - loss: 0.3170 - val_accuracy: 0.7069 - val_loss: 0.9132
Epoch 4/10
287/287 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9205 - loss: 0.2326 - val_accuracy: 0.7029 - val_loss: 1.0954
Epoch 5/10
287/287 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9490 - loss: 0.1488 - val_accuracy: 0.7176 - val_loss: 1.1738
Epoch 6/10
287/287 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9738 - loss: 0.0913 - val_accuracy: 0.7147 - val_loss: 1.3237
Epoch 7/10
287/287 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9833 - loss: 0.0584 - val_accuracy: 0.7176 - val_loss: 1.5628
Epoch 8/10
287/287 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9933 - loss: 0.0337 - val_accu

In [8]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"STF-IDF Test Accuracy: {accuracy:.4f}")


116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7009 - loss: 1.9539
STF-IDF Test Accuracy: 0.7061
